In [1]:
import torch


In [24]:
# class model_test(torch.nn.Module):
#     def __init__(self):
#         super(model_test).__init__()
import torch.nn as nn

conv1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3, stride=1, dilation=2)
conv1.weight.data.fill_(1)
x = torch.tensor([[1, 2, 3, 4, 5, 6, 7]], dtype=torch.float32)
output = conv1(x)
print(x)
print(output)
print(conv1.weight)

tensor([[1., 2., 3., 4., 5., 6., 7.]])
tensor([[ 8.4813, 11.4813, 14.4813]], grad_fn=<SqueezeBackward1>)
Parameter containing:
tensor([[[1., 1., 1.]]], requires_grad=True)


In [3]:
import torch.nn.functional as F
x = torch.randn(1,1,10)
x = F.pad(x, (2,0))
print(x)

tensor([[[ 0.0000,  0.0000, -0.9219, -0.1989, -2.2070,  2.0648, -0.0259,
          -1.2274, -0.7808,  1.9798,  0.4022, -0.4447]]])


# Causal Convolution with dilation
* implemented using padding the input with preceeding 0 padding and none at the end

In [30]:
class CausalConv1D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, dilation=1):
        super(CausalConv1D, self).__init__()
        self.kernel_size = kernel_size
        self.stride = stride
        self.dilation = dilation
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, stride, dilation=dilation)
        self.conv.weight.data.fill_(1)
        self.conv.bias.data.fill_(0)
        print(self.conv.weight)
    
    def forward(self, x):
        padding = (self.kernel_size - 1)*self.dilation
        x = F.pad(x, (padding, 0))
        print(x)
        return self.conv(x)
    

In [31]:
causalconv = CausalConv1D(1, 1, 3, dilation=1)
x = torch.tensor([[1, 2,3, 4, 5]], dtype=torch.float32)
output = causalconv.forward(x)
print("input: ", x)
print("output: ", output)
# print(causalconv.)
causalconvdil2 = CausalConv1D(1, 1, 3, dilation=2)
output2 = causalconvdil2.forward(output)
print("output2 :", output2)

Parameter containing:
tensor([[[1., 1., 1.]]], requires_grad=True)
tensor([[0., 0., 1., 2., 3., 4., 5.]])
input:  tensor([[1., 2., 3., 4., 5.]])
output:  tensor([[ 1.,  3.,  6.,  9., 12.]], grad_fn=<SqueezeBackward1>)
Parameter containing:
tensor([[[1., 1., 1.]]], requires_grad=True)
tensor([[ 0.,  0.,  0.,  0.,  1.,  3.,  6.,  9., 12.]],
       grad_fn=<ConstantPadNdBackward0>)
output2 : tensor([[ 1.,  3.,  7., 12., 19.]], grad_fn=<SqueezeBackward1>)
